# 03 – V2 Custom CNN on Raw Images

In this notebook we train a lightweight 2-convolution CNN on 64 × 64 RGB images.  
Goal: beat the MLP accuracy from notebook 02 and inspect early CNN performance.


In [ ]:
import numpy as np, matplotlib.pyplot as plt
from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

from src.data_loader import load_images
from src.model_cnn     import build_cnn
from src.compile_utils import compile_model, early_stop
from src.plotting      import plot_history
from src.evaluation    import evaluate



## 1  Load & preprocess
(Replace the demo paths with your real dataset folders.)


In [ ]:
folder_paths = [
    "../data/sample_images/recyclable",        # TODO replace
    "../data/sample_images/non_recyclable"     # TODO replace
]
class_names  = ["recyclable", "non-recyclable"]
target_size  = (64, 64)

X, y, ignored = load_images(folder_paths, class_names, target_size)
print(f"Loaded {X.shape}  |  ignored {ignored}")


### Train / validation / test split & one-hot encoding


In [ ]:
# encode labels
le = LabelEncoder()
y_int = le.fit_transform(y)
y_hot = to_categorical(y_int, num_classes=len(le.classes_))

# split (70 train / 20 val / 10 test)
X_temp, X_test, y_temp, y_test = train_test_split(
    X, y_hot, test_size=0.10, random_state=42, stratify=y_hot)

X_train, X_val, y_train, y_val = train_test_split(
    X_temp, y_temp, test_size=0.22, random_state=42, stratify=y_temp)  # 0.22→exact 0.7/0.2

print("Train:", X_train.shape, "Val:", X_val.shape, "Test:", X_test.shape)

# scale to [0-1]
X_train = X_train.astype("float32") / 255.0
X_val   = X_val.astype("float32")   / 255.0
X_test  = X_test.astype("float32")  / 255.0


## 2  Build & compile the CNN
Architecture:  
* Conv 32 → MaxPool  
* Conv 64 → MaxPool  
* Flatten → Dense 128 + Dropout 0.5 → Softmax 2  


In [ ]:
cnn = build_cnn(shape=(64,64,3), classes=len(le.classes_))
cnn = compile_model(cnn, lr=1e-3, loss="categorical_crossentropy")
cnn.summary()


## 3  Train


In [ ]:
H = cnn.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=25,
    batch_size=32,
    callbacks=[early_stop(patience=4)],
    verbose=2
)


## 4  Learning curves


In [ ]:
plot_history(H)


## 5  Evaluation on test set


In [ ]:
cm = evaluate(cnn, X_test, y_test, labels=le.classes_)


## 6  Discussion & next steps
* CNN test accuracy = … → improvement of ~X % over MLP.  
* Confusion matrix shows residual confusion mainly in **non-recyclable** class.  
* To push performance further we will:  
  1. **Add data augmentation** (Notebook 05).  
  2. Try **transfer learning (ResNet50)** (Notebook 04) for richer features.  

Proceed to the next experiment!
